![alt text](img/LM.png)
# Kurs: Deep Learning, Text Mining i XAI w Pythonie

## Autor: Piotr Ćwiakowski

### Lekcja 6. NLP: Uczenie nadzorowane i nienadzorowane

# Zadanie 1.

Przygotuj macierz DTM dla polskiego korpusu zapisów obrad Rady Polityki Pieniężnej znajdujących się w folderze `minutes_nbp_pl`. W tym celu:

1) Wczytaj dane pdf do jednej listy,

In [16]:
# Pakiety podstawowe
import re
import os
import PyPDF2
from os import listdir
import string
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import islice

# Pakiety do NLP
import nltk # nltk.download() 
import spacy
from nltk.corpus import stopwords # nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob, Word
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from wordcloud import WordCloud, STOPWORDS
from spacy import displacy

import Levenshtein as lev
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [17]:
sp = spacy.load('pl_core_news_sm') #wczytanie modelu językowego
stop_words = stopwords.words('english') # Stopwords po angielsku

# Przydatne polecenie:
# !python -m spacy validate
# https://spacy.io/api/cli

In [ ]:
# lista_plikow = listdir('dataTM/minutes_nbp_pl/')
# minutes = []

# for plik in lista_plikow[0:4]:
#     print(plik)
#     with open(r'dataTM/minutes_nbp_pl/' + plik, mode='rb') as fp:
#         pdf_doc = PyPDF2.PdfFileReader(fp)
#         doc = ''
#         for i in range(pdf_doc.numPages):
#             page_to_print = pdf_doc.getPage(i)
#             doc += page_to_print.extractText() # docs.append szybsze, ale potem i tak trzeba będzie skleić teksty.a
#             # Usuńmy entery pomiędzy literami:
# #             doc=re.sub(r'(\w)\n(\w)', '\\1\\2', doc)
#             # Usuńmy wszystkie entery
#             doc=re.sub(r'\n', '', doc)
#         minutes.append(doc)

In [36]:
with open('minutes_pl.p', 'rb') as fp:
    minutes = pickle.load(fp)

In [19]:
minutes[0]

'   1/4  444444444444444444444444444444444444444444444Opis  dyskusji  na  posiedzeniu  decyzyjnym  Rady  Polityki Pieniężnej w dniu 17 marca 2020 r. Dyskutując na temat bieżącej sytuacji gospodarczej członkowie Rady zwracali uwagę, że  wskutek  pandemii  COVID-19  oraz  działań  podejmowanych  przez wiele  państw zmierzających  do  ograniczenia  jej  dalszego  rozprzestrzeniania się, koniunktura  w gospodarce światowej istotnie się osłabiła. W szczególności wyraźnie spowolniła dynamika aktywności gospodarczej w sektorze usług, pogorszyły się nastroje konsumentów, a wiele firm na świecie czasowo wstrzymało produkcję . W tych warunkach silnie wzrosła też niepewność dotycząca perspektyw wzrostu PKB w  gospodarce światowej oraz  nastąpiło wyraźne pogorszenie  nastrojów  na  rynkach finansowych. Silnie spadły  ceny wielu  surowców,  w  tym  ropy  naftowej,  oraz nastąpiła deprecjacja walut części gospodarek wschodzących . Wskazywano,  że choć wedle  oficjalnych  danych w  Chinach rozprzestr

2) Oczyść dane, usuń stopwords (w tym też takie specyficzne dla tego korpusu) i wykonaj lematyzację,

In [34]:
stop_words = pd.read_csv('stopwords_pl.txt', encoding='utf8', header=None)[0].to_list()

# minutes = minutesCopy.copy()

# Tokenizacja po białych znakach (spacjach, enterach, etc.)
for i, doc in enumerate(minutes):
    # usuwanie podwójnych spacji oraz przede wszystkim '\s '
    doc = re.sub(r'\s+', ' ', doc)
    # Zamiana liter na małe
    doc = doc.lower()
    # Usuwanie liczb (opcjonalne)
    doc = re.sub(r'\d+', '', doc)
    # Dzielenie tekstu na słowa (tokeny)
    minutes[i] =  doc.split()
    # Usuwanie stopwords
    minutes[i] = [word for word in minutes[i] if word not in stop_words and word != '']
    # Usuwanie znaków interpunkcyjnych (trzeba się zastanowic, szczegolnie po angielsku)
    table = str.maketrans('', '', string.punctuation)
    minutes[i] = [w.translate(table) for w in minutes[i]]
    
# https://machinelearningmastery.com/clean-text-machine-learning-python/
# https://stackabuse.com/python-for-nlp-tokenization-stemming-and-lemmatization-with-spacy-library/bb

from nltk.probability import FreqDist

# Znajdźmy najpopularniejsze słowa przed tokenizacją
fdist = FreqDist(sum(minutes, []))
fdist.plot(30,cumulative=False)
plt.show()

for i, minute in enumerate(minutes):
    doc = sp(' '.join(minute))
    minutes[i] = [token.lemma_ for token in doc]

# Znajdźmy najpopularniejsze słowa po tokenizacji (w poszukiwaniu nowych stopwords)
fdist = FreqDist(sum(minutes, []))
fdist.plot(30,cumulative=False)
plt.show()

# Usuńmy stopwords
for i, doc in enumerate(minutes):
    minutes[i] = [word for word in minutes[i] if word not in ['rada', 'stopa', 'członek', '–', 'nbp', '', ' ']]

# Znajdźmy najpopularniejsze słowa po usunięciu stopwords
fdist = FreqDist(sum(minutes, []))
fdist.plot(30,cumulative=False)
plt.show()

3) policz macierz DTM dla wag TF-IDF

4) Wykonaj feature engineering dla korpusu - dodatkowo policz datę dla każdego dokumentu (można to wyciągnąć z nazwy) i zapisz jako osobną kolumnę w formie TF-IDF.

5) dla danych licznościowych (ang. *count*) policz dla kilku najważniejszych słów ("popyt", "wzrost", "inflacja", "polska", "płace", "niepewnosć") zaprezentuj na wykresie zmiany w częstościach w czasie. 

# Zadanie 2.

Wczytaj recenzje filmów znajdujące się w folderze `Movie reviews` i uporządkuj je do postaci DTM z wagami TF. Następnie wyeksportuj plik do formatu binarnego (pickle). 